## Lab 1 - Basic Numeric Tools

* give a situation that requires finding root of a function.  apply best root finder and solve.
* give a pathological function, look for root.
* code trapezoid rule
* Do 2-2 (first couple of functions?).  First find soln as function of # slices.  Then find "accuracy", using the value found above as "correct" value.  Plot as in Ch. 2 Integration notebook.  Determine how accuracy scales with dx (as quoted in book).
* Whiteboards: show how to use Taylor explansion to find eqn for 3-point and 5-point formulae for derivative.
* write a derivative function that acts on data: calc first and second derivative (e.g. to use * to find vel and accel from position.) (e.g. 2-7&2-8)
* examine difference between 3-pt and 5-pt routines on various functions/data?


In [1]:
%pylab

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


## Root Finding

Here are a couple of situations where you can use root finding to find an answer.  Pick an appropriate root finder for each case. In a comment, explain briefly why you chose the one you did. 

#### Mortgage Refinance

People usually borrow money from the bank to buy a house, which is called a mortgage.  When you have a mortgage you pay a fixed amount each month.  Your monthly payment was determined at the beginning of the loan, based on the borrowed amount, the length of time for the loan (eg. the number of payments), and the interest rate.  When interest rates drop during the life of a mortgage, you can "re-finance", which means you essentially borrow money to pay off your existing mortgage.  You then make monthly payments on the new loan. The advantage is that the new payments are lower because the interest rate is lower.  The disadvantage is that you are often charged a flat fee ("closing costs") to make the new loan.  So, the accumulated cost of your current mortgage is your current monthly payment times the number of months:

Cost_old(N) = payment_old * N

while the accumulated cost of the new mortgage would be your new monthly payment times the number of months *plus* the fixed closing cost:

Cost_new(N) = payment_new * N + closing.

Initially, Cost_new will be larger than Cost_old, because of the closing cost.  But over time, the accumulated cost of the new mortgage will be less than teh old mortgage, because of the lower monthly payments.  (Plot each of these functions to convince yourself this is true.)  For the following parameters, find your "break even time", the number of payments at which the new mortgage starts to be less than the old mortgage: 
* Old monthly payment = $ 2,200

* New monthly payment = $ 1,600

* Closing costs = $ 5,000


In [28]:
# Solution
#
# determine break-even time for Mortgage refinance.
# The function to find the root of is the difference between 
# the cumulative cost of the old mortgage and the cumulative 
# cost of the new mortgage plus the fixed closing costs:
# NetSavings = Cost_old - Cost_new
# 
# how to pick root finder: 
#  * Could plot the two functions separately and find the root approximately,
#  to determine bracketing start values
#  * Could determine the derivative of the function, d (NetSavings)/dN = payment_old-payment_new
#  use this for derivative in Newton's method.  
#  * Could just let scipy's secant method go with one start point.

# import root finders from scipy
import scipy.optimize as opt

# parameters of problem, for easy changing
payment_old= 2200
payment_new= 1600
ClosingCosts = 5000

# define cost functions
def Cost_old(N):
    return payment_old*N
def Cost_new(N):
    return payment_new*N+ClosingCosts
# define function to root find
def NetSavings(N):
    return Cost_old(N)-Cost_new(N)
def NetSavingsD(N):
    return payment_old-payment_new

# plot functions
Month=[]
OldCost=[]

#Find root

root_newton=opt.newton(NetSavings, 1, NetSavingsD)
root_b=opt.brentq(NetSavings, 1, 24)
print("break-even time: {} months (Newton), {} months (brentq)".format(root_newton, root_b))
figure()
Month=[x for x in range(1,24)]
OldCost=[Cost_old(x) for x in range(1,24)]
NewCost=[Cost_new(x) for x in range(1,24)]
plot(Month, OldCost, Month, NewCost)
xlabel="Month"
ylabel="Cost"
ylim(0,50000)
legend(['Old','New'],loc=4)
ans_str="{:.3} months (Newton)\n{:.3} months (brentq)".format(root_newton, root_b)
annotate(ans_str,xy=(3, 40000))


break-even time: 8.333333333333332 months (Newton), 8.333333333333332 months (brentq)


#### Vertical drop with air resistance

The vertical distance dropped in the presence of air resistance is given by:

$ y = \frac{v_{term}^2}{g} \ln ( \cosh ( \frac{gt}{v_{term}} )) $

where $ v_{term} = \sqrt{g m / c}$.  For m=50 kg and c ~ 0.25,
find the time to drop 20 meters.

In [36]:
# Solution
#
# this version depends on global variables

# define parameters
g=9.8
m=50
c=0.25
H= 20

# define functions
def v_term(m, c=0.25, g=9.8):
    return sqrt(g*m/c)

# def y_ar(t, v_term):
#    ans=(v_term**2/g)*log(cosh(g*t/(v_term)))
#    return ans
def y_ar(t):
    ans=(v_t**2/g)*log(cosh(g*t/(v_t)))
    return ans
def y_ar_root(t):
    return H-y_ar(t)

# calculate v_term
v_t=v_term(m, c)
print("v_term",v_t)

# Call Newton with secant method
time_to_drop = opt.newton(y_ar_root, 1)
# Call brentq
print("Time to drop {} meters is {} seconds".format(H, time_to_drop))

v_term 44.2718872424
Time to drop 20 meters is 2.054139154499366 seconds


In [38]:
# Solution
# This one passes parameters into functions to protect against
# possible collision with local variable in root-solving functions.
#
# define parameters
g=9.8
m=50
c=0.25
H= 20

# define functions
def v_term(m, c=0.25, g=9.8):
    return sqrt(g*m/c)

# def y_ar(t, v_term):
#    ans=(v_term**2/g)*log(cosh(g*t/(v_term)))
#    return ans
def y_ar(t,m,c,g):
    ans=(v_term(m,c,g)**2/g)*log(cosh(g*t/(v_term(m,c,g))))
    return ans
def y_ar_root(t,m,c,g,H):
    return H-y_ar(t,m,c,g)

# calculate v_term
v_t=v_term(m, c)
print("v_term:",v_t)

# Call Newton with secant method
time_to_drop = opt.newton(y_ar_root, 1, args=(m,c,g,H))
# Call brentq
print("Time to drop {} meters is {} seconds".format(H, time_to_drop))


v_term: 44.2718872424
Time to drop 20 meters is 2.054139154499366 seconds


## Integration

#### Trapezoid Method
Begin by looking at p. 61 in the text.  Show how to get Eq. 2.6 from Eq. 2.5.
Write a function that implements Eq. 2.6 for the trapezoid method of integration.  As in the simple integration, your function should take a list of function values and an interval dx. Test your routine on a function for which you know the integral.

In [52]:
# Solution
"""
Function to implement the trapezoid method to calculate the integral.
Takes a list of function values and the slice size dx.
"""
def int_trap(f, dx):
#    print("sum = {:.2}".format(sum(f[1:-1])))
#    print("f01 = {:.2}".format(f[0]+f[-1]))    
    return ((f[0]+f[-1])/2+sum(f[1:-1]))*dx

In [65]:
# Solution
# test trapezoid method integral function


#testf=full(10,1)
#lower and upper limits
a=0
b=10
N=100   # number of slices
dx= (b-a)/N
# try the constant function, integrated from 0 to 1
# testf=empty(N+1); testf.fill(2)
# try a triangle
#testf=[2*dx*j for j in range(N+1)]
# try sin
#testf=[sin(j*dx) for j in range(N+1)]
testf = [j*dx*sin(j*dx)**2 for j in range(N+1)]
#print(testf)
ans=int_trap(testf,dx)
print("dx= {:.4}, ans={:.4}".format(dx, ans))

dx= 0.1, ans=22.8


### Comparing Integration methods: Problem 2-2
Do problem 2-2.  It asks you to compare the simple integration, your trapezoid integration, and the built-in Simpson's methods. It asks you to explore the behavior of the method on several functions as you vary the size of dx (or number of steps N).  Probably the best way to do this is to loop over the number of steps, calculate the value of the integral, and plot the results so you can see how they vary with N.  Intuitively, we expect the value of the integral to converge to a value as we increase the number of slices.  The question asks you to determine about how many slices are needed.

Once you've established what the asymptotic value of the integral (e.g. the "true" value), you can do what we showed in class on Wed to plot the accuracy vs. the number of slices. From these plots you can determine how the accuracy depends on N (or dx).

To help you do this, go back to the "Chapter 2 - Integration" notebook and look at the plots of accuracy vs N for the Trapezoid and Simpson's methods.  Which increases accuracy faster?  How fast does each go -- how do we describe them in the text?  Since these are log plots, the slope tells us the *power dependence* of accuracy on N.  That is, a slope of -2 means accuracy goes as $N^{-2}$. With this knowledge, characterize how the accuracy depends on N for each integration method you use here.  Compare to what is written on p. 65 in the text.

## Differentiation

#### Deriving the 3-point and 5-point relations for derivative
a. On a whiteboard, write the Taylor expansion for a function $f(x)$ around $x_0$.  

b. To turn this into a discrete expression, useful for working with datapoints, think of $x_0$ as $x_i$ and $x$ as $x_{i+1}$, the next value of x in the discrete set.  Translate your expression for the Taylor expansion into this notation, and simplify it using $\Delta x = x_{i+1}-x_i$.

c. Solve this expression for $f^{\prime}$. Ignoring the terms is $\Delta x$ and higer, this is the first order approximation to the derivative. The error we're making is all those terms we ignored, which is on the order of $\Delta x$.

d. To do better, write out the Taylor expansion in discrete form for $f(x_{i-1})$.  What you are doing now is thinking about the point *before* $x_i$. ($x_i$ is still $x_0$).

e. Now, **subtract** your Taylor explansion for $f(x_{i-1})$ from the one for $f(x_{i+1})$.  Solve this expression for $f^{\prime}$.  Now the biggest error term is proportional to $\Delta x^2$. This is the "3-point derivative" formula.  (Why "3-point"?).

f. To find the "5-point derivative" formula, consider the Taylor expansion for $f(x_{i\pm 2})$, subtracting one from the other. See page 67 for the final steps, but do them on your whiteboard and confirm the final result. 

g. What's the advantage of the 5-point over the 3-point method?  What's a disadvantage, especially when acting on data?

#### Implementing the 3-point derivative method

(Problem 2-7) Write a function that, given a list of function values $f_i$ and the spacing
$dx$ between them, returns a list of values of the first derivative of the
function. Test your function by giving it a list of known function values
for $\sin(x)$ and making a graph of the differences between the output
of the function and $\cos(x)$.

(Problem 2-8) Write a function that, given a list of function values $f_i$ and the spacing
$dx$ between them, returns a list of values of the second derivative of the
function. Test your function by giving it a list of known function values
for $\sin(x)$ and making a graph of the differences between the output
of the function and $-\sin(x)$.

Use your two derivative functions to calculate velocity and acceleration from a list of time and position values (for example, from the motion detector in LoggerPro.)